<a href="https://colab.research.google.com/github/wqiu96/MA574_Portfolio-Valuation-and-Risk-Management/blob/master/src/regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import os

In [2]:
!git clone https://github.com/wqiu96/MA574_Portfolio-Valuation-and-Risk-Management.git

Cloning into 'MA574_Portfolio-Valuation-and-Risk-Management'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 560 (delta 51), reused 0 (delta 0), pack-reused 437
Receiving objects: 100% (560/560), 3.35 MiB | 23.17 MiB/s, done.
Resolving deltas: 100% (223/223), done.


In [3]:
cd MA574_Portfolio-Valuation-and-Risk-Management/src/project4/

/content/MA574_Portfolio-Valuation-and-Risk-Management/src/project4


In [0]:
#Load data
os.chdir('/content/MA574_Portfolio-Valuation-and-Risk-Management/src/project4/')
file_chdir = os.getcwd()
filecsv_list = []
for root,dirs,files in os.walk(file_chdir):
  for file in files:
    if os.path.splitext(file)[1] == '.xlsx':
      filecsv_list.append(file)
R = pd.DataFrame()
for xls in filecsv_list:
  R[xls.strip('.xlsx')] = pd.read_excel(xls)['Last Price']

In [55]:
files

['GDP.xlsx',
 'GOVT PURCHASE and INVESTMENT.xlsx',
 'US URBAN CPI.xlsx',
 'Core Price Consumption.xlsx',
 'US FEDERAL FUNDS TARGET RATE.xlsx',
 'Private Domestic Investment.xlsx',
 'US Trade Balance.xlsx']

In [62]:
Y = R['GDP'].values
Par = R.copy()
Par['GDP'] = np.ones(Par.shape[0])
Par.columns = ['Beta_0', 'GOV_INV', 'CPI','CONSUMPTION','FED_RATE','PRIVATE_INV','TRADE']
data = Par.join(R['GDP'])
X = Par.values
B = np.dot(np.dot(np.linalg.inv(np.dot(X.T,X)),X.T),Y)
B

array([ 1.57797457,  0.25009288, -1.47665857,  0.78230875,  0.23154418,
        0.89211681,  0.00985919])

In [73]:
# load packages
import statsmodels.api as sm
from statsmodels.formula.api import ols
# Ordinary Least Squares (OLS) model
model = ols('GDP ~ GOV_INV + CPI + CONSUMPTION + FED_RATE + PRIVATE_INV + TRADE', data=data).fit()
model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    GDP   R-squared:                       0.687
Model:                            OLS   Adj. R-squared:                  0.630
Method:                 Least Squares   F-statistic:                     12.05
Date:                Sat, 09 Nov 2019   Prob (F-statistic):           3.88e-07
Time:                        22:00:07   Log-Likelihood:                -68.126
No. Observations:                  40   AIC:                             150.3
Df Residuals:                      33   BIC:                             162.1
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       1.5780      0.831      1.898      0.066      -0.114       3.269
GOV_INV         0.2501      0.111      2.256      0.031       0.025       0.476
CPI            -1.4767      1.521     -0.971      0.339      -4.571       1.618
CONSUMPTION     0.7823      0.516      1.517      0.139      -0.267       1.831
FED_RATE        0.2315      0.414      0.559      0.580      -0.611       1.074
PRIVATE_INV     0.8921      0.135      6.603      0.000       0.617       1.167
TRADE           0.0099      0.117      0.084      0.933      -0.228       0.248
==============================================================================
Omnibus:                        0.651   Durbin-Watson:                   2.579
Prob(Omnibus):                  0.722   Jarque-Bera (JB):                0.750
Skew:                          -0.254   Prob(JB):                        0.687
Kurtosis:                       2.561   Cond. No.                         20.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""